In [1]:
from simulate_s2s_uns2uns_model import simulate_steps
import analyse_results
import pandas as pd


#Generate metrics/model information for dictionary of networks
#and their detections data
def generate_model_data(ntwkDict):
    #metrics information needed
    model_data = ['P_s2s','P_u2s','P_visible','seen','seen2seen',\
                 'seen2unseen','unseen','unseen2seen','unseen2unseen']
    model_names = list(ntwkDict.keys())
    df = pd.DataFrame(index=model_data,columns=model_names)
    for model in model_names:
        #compute data for each litter
        ntwkDF = ntwkDict[model]
        
        ntwkDF.loc[:,'P_s2s'] = \
        ntwkDF['seen2seen']/(ntwkDF['seen2seen']+ntwkDF['seen2unseen'])
        
        ntwkDF.loc[:,'P_u2s'] = \
        ntwkDF['unseen2seen']/(ntwkDF['unseen2seen']+ntwkDF['unseen2unseen'])
        
        ntwkDF.loc[:,'P_visible'] = ntwkDF['seen']/ntwkDF['visible']

        #summarize data into new df
        for data in model_data:
            if 'P_' in data:
                df.loc[data,model] = f'${ntwkDF[data].mean():.4f} \pm {ntwkDF[data].std():.4f}$'
            else:
                df.loc[data,model] = f'{int(ntwkDF[data].sum())}'
    return df
    

In [2]:
def read_probability_file(fps,nafilter):
    means = pd.read_csv(f"probabilities/{fps}fps/TPandFN_probability_means_nafilter_{nafilter}.csv",
                       header=0,index_col=0)
    stddev = pd.read_csv(f"probabilities/{fps}fps/TPandFN_probability_stddev_nafilter_{nafilter}.csv",
                       header=0,index_col=0)
    ntwkDict = {}
    tex_df = pd.DataFrame()
    for ntwk in means.columns:
        ntwkDict[ntwk] = ((means.loc['P_s2s',ntwk],stddev.loc['P_s2s',ntwk]),\
                          (means.loc['P_u2s',ntwk],stddev.loc['P_u2s',ntwk]))
        tex_df.loc['P_s2s',ntwk] = f"${means.loc['P_s2s',ntwk]:.4f} \pm {stddev.loc['P_s2s',ntwk]:.4f}$"
        tex_df.loc['P_u2s',ntwk] = f"${means.loc['P_u2s',ntwk]:.4f} \pm {stddev.loc['P_u2s',ntwk]:.4f}$"
        tex_df.loc['P_visible',ntwk] = f"${means.loc['P_visible',ntwk]:.4f} \pm {stddev.loc['P_visible',ntwk]:.4f}$"
    return means,stddev,ntwkDict,tex_df

In [3]:
def simulate_and_analyse_ntwk_data(ntwkDict):
    simDict = {}
    for ntwk in ntwkDict:
        print(ntwk)
        p_s2s,p_u2s = ntwkDict[ntwk]
        simDict[ntwk] = simulate_steps(p_s2s,p_u2s,mode,nlitter,tsteps)
    return generate_model_data(simDict)

In [8]:
nlitter = 100
tsteps = 100
mode='mean'
fps=1
nafilter='0'
means,stddev,ntwkDict,tex_df = read_probability_file(fps,nafilter)
sim_data = simulate_and_analyse_ntwk_data(ntwkDict)

mSSD-124
 99                                                                                                                                                                                                                            
mSSD-220
 99                                                                                                                                                                                                                                                                   
yolov3-tiny-128
 99                                                                                                                                                                                                                                                                                       
yolov3-tiny-224
 99                                                                                                                                                                                 

In [9]:
probabilities=['P_u2s','P_s2s','P_visible']
display('experiment data',tex_df.loc[probabilities,:])
display('simulated data',sim_data.loc[probabilities,:])

'experiment data'

,mSSD-124,mSSD-220,yolov3-tiny-128,yolov3-tiny-224
P_u2s,$0.3827 \pm 0.3434$,$0.4916 \pm 0.4081$,$0.1310 \pm 0.2341$,$0.1983 \pm 0.2976$
P_s2s,$0.4820 \pm 0.4659$,$0.6352 \pm 0.4053$,$0.1300 \pm 0.3170$,$0.2037 \pm 0.3648$
P_visible,$0.4185 \pm 0.2923$,$0.6042 \pm 0.2852$,$0.1214 \pm 0.2032$,$0.2039 \pm 0.2712$


'simulated data'

,mSSD-124,mSSD-220,yolov3-tiny-128,yolov3-tiny-224
P_u2s,$0.3795 \pm 0.0705$,$0.5000 \pm 0.0711$,$0.1338 \pm 0.0336$,$0.2012 \pm 0.0467$
P_s2s,$0.4737 \pm 0.0867$,$0.6171 \pm 0.0661$,$0.1362 \pm 0.0902$,$0.1898 \pm 0.0931$
P_visible,$0.4183 \pm 0.0606$,$0.5662 \pm 0.0569$,$0.1342 \pm 0.0310$,$0.1992 \pm 0.0418$
